В этом задании будет использоваться датасет digits из sklearn.datasets. 
Оставьте последние 25% объектов для контроля качества, 
разделив X и y на X_train, y_train и X_test, y_test.
Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа,
а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. 
Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта 
составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), 
затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, 
затем по второму), а затем брать первый элемент (с наименьшим расстоянием).
Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). 
Подумайте, как можно легко улучшить получившееся время работы. 
Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, 
как разбить пространство признаков на части и сделать структуру данных, 
которая позволит быстро искать соседей каждой точки. 
За выбор метода поиска ближайших соседей в KNeighborsClassifier 
из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных,
вам может быть интересно познакомиться со структурами данных ball tree и kd tree.
Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.


Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. 
Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. 
Эта доля — ответ в задании 2. 
Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, 
пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, 
но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from math import sqrt
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import stats
import time
import warnings
warnings.filterwarnings('ignore')

In [15]:
class KNN_Classifier:

    def __init__(self):
        self.scaler = MinMaxScaler()
        
    def scale(self, X):
        X = self.scaler.fit_transform(X)
        return X

    def add_class(self, X_train, y_train):
        X_train2 = np.zeros((X_train.shape[0], X_train.shape[1]+1), dtype = float)
        X_train2[:, :-1] = X_train[:,:].copy()    
        X_train2[:, -1] = y_train
        return X_train2
    
    @staticmethod    
    def euclideanDistance(instance1, instance2):
        distance = 0
        length = len(instance1) - 1
        for x in range(length):
            distance += pow((instance1[x] - instance2[x]), 2)
        return sqrt(distance)
    
    def euclidean_distance(self, row1, row2):
        distance = 0.0
        for i in range(len(row1)-1):
            distance += (row1[i] - row2[i])**2
        return sqrt(distance)
    
    def get_neighbors(self, train, test_row, k):
        distances = []
        for train_row in train:
            dist = self.euclidean_distance(test_row, train_row)
            distances.append([train_row, dist])
        distances.sort(key=lambda x: x[1])
        neighbors = []
        for i in range(k):
            neighbors.append(distances[i][0])
        return neighbors
    
    @staticmethod
    def get_neighbors1(train, test_row, k):
        distances = []
        dst = []
        mn = 10000
        for train_row in train:
            dist = KNN_Classifier.euclideanDistance(test_row, train_row)
            distances.append([train_row, dist])
            dst.append(dist)
            if dist < mn:
                mn = dist
        if k == 1:
            neighbors = [distances[dst.index(mn)]]
        else:
            distances.sort(key=lambda x: x[1])
            neighbors = []
            for i in range(k):
                neighbors.append(distances[i][0])
        return neighbors
    
    @staticmethod
    def get_neighborsK(train, test_row, k):
        dst = []
        mn = [10000 for x in range(k)]
        for train_row in train:
            dist = KNN_Classifier.euclideanDistance(test_row, train_row)
            dst.append(dist)
            mn.append(dist)
            mn.sort()
            mn = mn[:k]
        mn.sort()
        if k == 1:
            neighbors = [train[dst.index(mn[0])]]
        else:
            neighbors = []
            for i in range(k):
                neighbors.append(train[dst.index(mn[i])])
        return neighbors
    
    def find_center_clasters(self, X_train2):
        centers = []
        y = X_train2[:,-1]
        y = list(set(y))
        y.sort()
        for i in range(len(y)):
            xc = X_train2[X_train2[:,-1] == y[i]][:-1]
            v = [np.mean(xc[:,x]) for x in range(xc.shape[1])]+[y[i]]
            centers.append(v)
        return centers 
    
    def find_center_clasters2(self, X_train2):
        centers = []
        y = X_train2[:,-1]
        y = list(set(y))
        y.sort()
        for i in range(len(y)):
            xc = X_train2[X_train2[:,-1] == y[i]][:-1]
            v = [np.median(xc[:,x]) for x in range(xc.shape[1])]+[y[i]]
            centers.append(v)
            v = [np.mean(xc[:,x]) for x in range(xc.shape[1])]+[y[i]]
            centers.append(v)
            v = [stats.mode(xc[:,0])[0][0] for x in range(xc.shape[1])]+[y[i]]
            centers.append(v)  
        return centers 
    
    def predict_knn(self, train, test_row, k):
        neighbors = self.get_neighbors(train, test_row, k)
        output_values = [row[-1] for row in neighbors]
        unique, counts = np.unique(output_values, return_counts=True)
        counter = dict(zip(unique, counts))
        res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
        res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)
        prediction = res_df.iloc[0,0]
        return prediction
    
    def predict_knn2(self, train, test_row, k):
        centers = self.find_center_clasters(train)
        neighbors = self.get_neighbors(centers, test_row, k)
        output_values = [row[-1] for row in neighbors]
        unique, counts = np.unique(output_values, return_counts=True)
        counter = dict(zip(unique, counts))
        res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
        res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)
        prediction = res_df.iloc[0,0]
        return prediction
    
    def predict_knn3(self, train, test_row, k):
        centers = self.find_center_clasters2(train)
        neighbors = self.get_neighbors(centers, test_row, k)
        output_values = [row[-1] for row in neighbors]
        unique, counts = np.unique(output_values, return_counts=True)
        counter = dict(zip(unique, counts))
        res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
        res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)
        prediction = res_df.iloc[0,0]
        return prediction
    
    def accuracy_metric(self, actual, predicted):
        correct = 0
        for i in range(len(actual)):
            if actual[i] == predicted[i]:
                correct += 1
        return round(correct / float(len(actual)) * 100.0, 2)
     
    def getAccuracy(self, testSet, predictions):
        correct = 0
        for x in range(len(testSet)):
            if testSet[x][-1] == predictions[x]:
                correct += 1
        return round((correct/float(len(testSet))) * 100.0, 2)
 
    def test1(self, X_train2, X_test2, k): 
        predictions=[]
        for i in tqdm(range(len(X_test2))):
            result = self.predict_knn(X_train2, X_test2[i], k)
            predictions.append(result)
        accuracy = self.getAccuracy(X_test2, predictions)
        print('Accuracy1: ' + repr(accuracy) + '%')
        accuracy = self.accuracy_metric(y_test, predictions)
        print('Accuracy2: ' + repr(accuracy) + '%')

    def test2(self, X_train2, X_test2, k): 
        predictions=[]
        for i in tqdm(range(len(X_test2))):
            result = self.predict_knn2(X_train2, X_test2[i], k)
            predictions.append(result)
        accuracy = self.getAccuracy(X_test2, predictions)
        print('Accuracy1: ' + repr(accuracy) + '%')
        accuracy = self.accuracy_metric(y_test, predictions)
        print('Accuracy2: ' + repr(accuracy) + '%')

    def test3(self, X_train2, X_test2, k): 
        predictions=[]
        for i in tqdm(range(len(X_test2))):
            result = self.predict_knn3(X_train2, X_test2[i], k)
            predictions.append(result)
        accuracy = self.getAccuracy(X_test2, predictions)
        print('Accuracy1: ' + repr(accuracy) + '%')
        accuracy = self.accuracy_metric(y_test, predictions)
        print('Accuracy2: ' + repr(accuracy) + '%')

### Загрузка датасета

In [16]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

### Создать экземпляр класса

In [17]:
knnc = KNN_Classifier()

### Нормализация данных

In [18]:
X = knnc.scale(X)

### Разбить на данные для обучения и валидации

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True, 
                                                    random_state=42, 
                                                    stratify=y)

### Добавить класс в выборку

In [20]:
X_train2 = knnc.add_class(X_train, y_train)
X_test2 = knnc.add_class(X_test, y_test)

### Самостоятельная реализация метода одного ближайшего соседа с евклидовой метрикой для задачи классификации

In [21]:
knnc.test1(X_train2, X_test2, k=1)

100%|██████████| 450/450 [00:27<00:00, 16.20it/s]

Accuracy1: 98.44%
Accuracy2: 98.44%


# Ответ 1: 98.4%

### Придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки.

##### Найду центры каждого кластера как среднее по каждой фиче и буду сравнивать с ними

In [22]:
knnc.test2(X_train2, X_test2, k=1)

100%|██████████| 450/450 [00:03<00:00, 148.96it/s]

Accuracy1: 87.56%
Accuracy2: 87.56%


##### Найду центры каждого кластера как (среднее, медиана, мода) по каждой фиче и буду сравнивать с ними

In [23]:
knnc.test3(X_train2, X_test2, k=1)

100%|██████████| 450/450 [00:34<00:00, 13.06it/s]

Accuracy1: 87.56%
Accuracy2: 87.56%


#### Качество уменьшилось, но скорость поиска намного выросла

### Стандартный поиск ближайшего соседа

In [24]:
%%time
neighbors = knnc.get_neighbors(X_train, X_train[0], k=1)

CPU times: user 62.9 ms, sys: 1.1 ms, total: 64 ms
Wall time: 63.2 ms


### Различные тесты ускорения поиска ближайшего соседа

In [25]:
%%time
neighbors = KNN_Classifier.get_neighbors1(X_train, X_train[0], k=1)

CPU times: user 66.3 ms, sys: 1.51 ms, total: 67.8 ms
Wall time: 66.9 ms


In [26]:
%%time
neighbors = KNN_Classifier.get_neighborsK(X_train, X_train[0], k=1)

CPU times: user 65 ms, sys: 1.51 ms, total: 66.5 ms
Wall time: 65.4 ms


### Задание 2

In [27]:
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)
y_pred=rf.predict(X_test)

accuracy = knnc.getAccuracy(X_test2, y_pred)
print('Accuracy1: ' + repr(accuracy) + '%')
accuracy = knnc.accuracy_metric(y_test, y_pred)
print('Accuracy2: ' + repr(accuracy) + '%')

Accuracy1: 97.11%
Accuracy2: 97.11%


# Ответ 2: 97.11%

##### Случайный лес показал худший результат, чем базовый алгоритм поиска ближайшего соседа